In [1]:
from dotenv import load_dotenv
import os, getpass
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint,HuggingFacePipeline
load_dotenv()  # Loads environment variables from .env file

# ignore all warnings in this notebook
import warnings
warnings.filterwarnings("ignore")

/workspaces/Langchain-Tutorial/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
huggingface_api_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if huggingface_api_key:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_api_key
    print("HUGGINGFACEHUB_API_TOKEN found in .env file.")

else:
    print("HUGGINGFACEHUB_API_TOKEN not found in .env file.")
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter Hugging Face API key: ")

HUGGINGFACEHUB_API_TOKEN not found in .env file.


## HuggingFaceEndpoint

**What do you mean by huggingface endpoint here?**

A Hugging Face Endpoint allows you to deploy and serve your models in the cloud, making it easy to integrate them into applications without worrying about the underlying infrastructure.


In [3]:
deepseek_llm_endpoint = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528",
    task="text-generation",
    max_new_tokens=512,
    provider="auto",  # let Hugging Face choose the best provider for you
)

hf_chat_model = ChatHuggingFace(llm = deepseek_llm_endpoint)

In [4]:
deepseek_result = hf_chat_model.invoke("A farmer has 17 sheep. All but 9 run away. How many are left?")
print(deepseek_result)

content='<think>\nI have this problem: A farmer has 17 sheep. All but 9 run away. How many are left?\n\nFirst, I need to understand what "all but 9" means. I think it means that all sheep except for 9 ran away. So, if all except 9 ran away, that means 9 are left.\n\nLet me read it again: "All but 9 run away." So, "all but" usually means "except for." For example, "all but one" means everyone except one.\n\nSo here, "all but 9" should mean all except 9. So, all sheep except 9 ran away, which means 9 did not run away and are left.\n\nThe farmer has 17 sheep. If all except 9 ran away, then 9 are left.\n\nBut let me double-check. If 9 ran away, then 17 minus 9 is 8 left, but the phrase is "all but 9 run away," which I think means that 9 are the ones that didn\'t run away.\n\nI can think in terms of who ran away. "All but 9 run away" means that all except 9 ran away, so 9 did not run away.\n\nTherefore, the number left should be 9.\n\nBut let me make sure with an example. Suppose there are 

In [6]:
print(deepseek_result.content)

<think>
I have this problem: A farmer has 17 sheep. All but 9 run away. How many are left?

First, I need to understand what "all but 9" means. I think it means that all sheep except for 9 ran away. So, if all except 9 ran away, that means 9 are left.

Let me read it again: "All but 9 run away." So, "all but" usually means "except for." For example, "all but one" means everyone except one.

So here, "all but 9" should mean all except 9. So, all sheep except 9 ran away, which means 9 did not run away and are left.

The farmer has 17 sheep. If all except 9 ran away, then 9 are left.

But let me double-check. If 9 ran away, then 17 minus 9 is 8 left, but the phrase is "all but 9 run away," which I think means that 9 are the ones that didn't run away.

I can think in terms of who ran away. "All but 9 run away" means that all except 9 ran away, so 9 did not run away.

Therefore, the number left should be 9.

But let me make sure with an example. Suppose there are 10 sheep, all but 3 run awa

In [7]:
import re

def extract_thinking(text):
    pattern = re.compile(r"<think>(.*?)</think>", re.IGNORECASE | re.DOTALL)
    matches = pattern.findall(text)
    if matches:
        return "\n\n".join(m.strip() for m in matches)
    return text

def extract_after_think(text):
    pattern = re.compile(r".*</think>(.*)", re.IGNORECASE | re.DOTALL)
    m = pattern.search(text)
    if m:
        return m.group(1).strip()
    return text

In [8]:
extract_thinking(deepseek_result.content)

'I have this problem: A farmer has 17 sheep. All but 9 run away. How many are left?\n\nFirst, I need to understand what "all but 9" means. I think it means that all sheep except for 9 ran away. So, if all except 9 ran away, that means 9 are left.\n\nLet me read it again: "All but 9 run away." So, "all but" usually means "except for." For example, "all but one" means everyone except one.\n\nSo here, "all but 9" should mean all except 9. So, all sheep except 9 ran away, which means 9 did not run away and are left.\n\nThe farmer has 17 sheep. If all except 9 ran away, then 9 are left.\n\nBut let me double-check. If 9 ran away, then 17 minus 9 is 8 left, but the phrase is "all but 9 run away," which I think means that 9 are the ones that didn\'t run away.\n\nI can think in terms of who ran away. "All but 9 run away" means that all except 9 ran away, so 9 did not run away.\n\nTherefore, the number left should be 9.\n\nBut let me make sure with an example. Suppose there are 10 sheep, all but

In [9]:
extract_after_think(deepseek_result.content)

'The phrase "all but 9" means all sheep except for 9. Therefore, if all except 9 sheep ran away, then 9 sheep are left.\n\nThe farmer started with 17 sheep, and after all but 9 run away, the number of sheep remaining is 9.'